In [8]:
#Prueba de actualizacion - Base de dato DNRPA
# Carga de librerias

import datetime
import mysql.connector
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import openpyxl
import pandas as pd
from datetime import datetime
from selenium.webdriver.support.ui import Select
import os

#Variables globales
host = '172.17.22.10'
user = 'Ivan'
password = 'Estadistica123'
database = 'prueba1'

#DF obtenido del sitio DRPNA
df_web = pd.DataFrame()

In [9]:
# Conexion con BDD 
conn = mysql.connector.connect( host=host, user=user, password=password, database=database)

#Instancia manejadora de BDD

cursor = conn.cursor()

In [10]:
#Busqueda por nombre de TABLA 

table_name = "dnrpa_parque_activo_nacion"

#Creacion de orden en lenguaje MYSQL
select_dates_query = "SELECT * FROM dnrpa_parque_activo_nacion"

#Obtencion del DATAFRAME - uso de Orden creada(MySQL) y Conexion(MySQL).
df = pd.read_sql(select_dates_query,con=conn)


#Busqueda del año mas grande (actual o anterior) --> Aclaracion por posible conflicto en los avisos por cada año nuevo
año_de_comparacion = int(df['Fecha'].max().year)

#Conversion a tipo de fecha - necesario para el tratamiento
df['Fecha'] = pd.to_datetime(df['Fecha'])

#Toma de año actual - Se usara para comparar los dataframes
df_año_actual = df[df['Fecha'].dt.year == año_de_comparacion]
print(df_año_actual)


         Fecha  Buenos_Aires  C_Autonoma_De_BSAS  Catamarca    Cordoba  \
156 2023-01-01     6556510.0           1596530.0   125583.0  1743920.0   
157 2023-02-01     6566750.0           1597140.0   125768.0  1746650.0   
158 2023-03-01     6578960.0           1599450.0   126006.0  1750200.0   
159 2023-04-01     6589720.0           1601070.0   126192.0  1753280.0   
160 2023-05-01     6601670.0           1604260.0   126412.0  1756830.0   
161 2023-06-01     6614240.0           1607330.0   126678.0  1760220.0   
162 2023-07-01           0.0                 0.0        0.0        0.0   
163 2023-08-01           0.0                 0.0        0.0        0.0   
164 2023-09-01           0.0                 0.0        0.0        0.0   
165 2023-10-01           0.0                 0.0        0.0        0.0   
166 2023-11-01           0.0                 0.0        0.0        0.0   
167 2023-12-01           0.0                 0.0        0.0        0.0   

     Corrientes     Chaco    Chubut  

C:\Users\Usuario\AppData\Local\Temp\ipykernel_13704\3661196741.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(select_dates_query,con=conn)


In [11]:
#Obtenicion de datos NRPA - Parque activo nacion

try:

    driver = webdriver.Chrome()
    driver.get('https://www.dnrpa.gov.ar/portal_dnrpa/boletines_estadisticos2.php')

    # Obtener la ventana actual
    ventana_actual = driver.current_window_handle
    
    elemento = driver.find_element(By.XPATH, '/html/body/div/section/article/div/div[2]/div/div/div/div/form/table/tbody/tr[2]/td[2]/select')

    # Localizar el elemento select por su nombre
    select_element = elemento.find_element(By.XPATH, '/html/body/div/section/article/div/div[2]/div/div/div/div/form/table/tbody/tr[2]/td[2]/select')

    # Crear un objeto Select
    select = Select(select_element)
    
    # O seleccionar por texto visible
    valor_deseado = '2023'
    select.select_by_visible_text('Año 2023')
    
    # Esperar un momento para que se abra la nueva pestaña
    driver.implicitly_wait(5)
    # Cambiar al contexto de la nueva pestaña
    for ventana in driver.window_handles:
        if ventana != ventana_actual:
            driver.switch_to.window(ventana)
    
    time.sleep(5)
    #↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓ ENCONTRAR Y TOMAR LOS DATOS ↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
    # Encontrar el elemento <div> con la clase 'grid'
    elemento_div = driver.find_element(By.CLASS_NAME, 'grid')

    # Encontrar la tabla dentro del elemento <div>
    elemento_tabla = elemento_div.find_element(By.TAG_NAME, 'table')

    # Obtener todas las filas de la tabla
    filas = elemento_tabla.find_elements(By.TAG_NAME, 'tr')

    # Lista para almacenar los datos de la tabla
    tabla_datos = []
    
    for fila in filas:
        # Obtener las celdas de cada fila, excluyendo la última columna y la última celda de encabezado
        celdas = fila.find_elements(By.TAG_NAME, 'th') + fila.find_elements(By.TAG_NAME, 'td')

        # Lista para almacenar los valores de cada fila
        fila_datos = []

        for celda in celdas:
            valor = celda.text
            if isinstance(valor, str):
                # Verificar si el valor comienza con un número
                if valor.strip() and valor[0].isdigit():
                    try:
                        # Reemplazar el punto decimal por una coma (si es necesario)
                        valor = valor.replace('.', '')
                        # Intentar convertir el valor a float
                        valor = float(valor)
                        print("valor1: ", valor)
                    except ValueError:
                        pass  # Mantener el valor original si no se puede convertir a float
            fila_datos.append(valor)
        print("aca: ", fila_datos)


        tabla_datos.append(fila_datos) 
    
    datos_sin_segunda_fila = tabla_datos[0:1] + tabla_datos[2:]
    # Transponer los datos utilizando pandas
    df = pd.DataFrame(datos_sin_segunda_fila)
    df_transpuesta = df.transpose()
    
    #Conversion de MESES a formato Y-M-D , tipo de dato: datetime
    print(df_transpuesta[0][1:])
    meses = df_transpuesta[0][1:]

    #Donde almacenamos las nuevas fechas
    nuevas_fechas = list()

    for i in range(1, len(meses)+ 1):

        if i < 10:
            fecha_str =  '01-0'+str(i)+"-"+ str(valor_deseado)
        else:
            fecha_str = '01-'+str(i)+"-"+ str(valor_deseado)

        fecha_str = datetime.strptime(fecha_str,'%d-%m-%Y').date()
        nuevas_fechas.append(fecha_str)

    #Reasignacion de fechas
    df_transpuesta[0][1:] = nuevas_fechas
    
    df_web = df_transpuesta

except Exception as e:
     # Manejar cualquier excepción ocurrida durante la carga de datos
     print(f"Registro automotor: Ocurrió un error durante la carga de datos: {str(e)}")
     conn.close()  # Cerrar la conexión en caso de error

aca:  ['Provincia / Mes', 'Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
aca:  []
valor1:  6556514.0
valor1:  6566752.0
valor1:  6578965.0
valor1:  6589720.0
valor1:  6601673.0
valor1:  6614240.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
aca:  ['BUENOS AIRES', 6556514.0, 6566752.0, 6578965.0, 6589720.0, 6601673.0, 6614240.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
valor1:  1596529.0
valor1:  1597143.0
valor1:  1599450.0
valor1:  1601073.0
valor1:  1604263.0
valor1:  1607331.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
aca:  ['CABA', 1596529.0, 1597143.0, 1599450.0, 1601073.0, 1604263.0, 1607331.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
valor1:  125583.0
valor1:  125768.0
valor1:  126006.0
valor1:  126192.0
valor1:  126412.0
valor1:  126678.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
aca:  ['CATAMARCA', 125583.0, 125768.0, 126006.0, 126192.0, 126412.0, 126678.0, 0

In [12]:
print(df_web)

print("""
     ********************************************************************
      """)

print(df_año_actual)

                 0             1          2          3          4   \
0   Provincia / Mes  BUENOS AIRES       CABA  CATAMARCA    CORDOBA   
1        2023-01-01     6556514.0  1596529.0   125583.0  1743924.0   
2        2023-02-01     6566752.0  1597143.0   125768.0  1746647.0   
3        2023-03-01     6578965.0  1599450.0   126006.0  1750200.0   
4        2023-04-01     6589720.0  1601073.0   126192.0  1753283.0   
5        2023-05-01     6601673.0  1604263.0   126412.0  1756829.0   
6        2023-06-01     6614240.0  1607331.0   126678.0  1760221.0   
7        2023-07-01           0.0        0.0        0.0        0.0   
8        2023-08-01           0.0        0.0        0.0        0.0   
9        2023-09-01           0.0        0.0        0.0        0.0   
10       2023-10-01           0.0        0.0        0.0        0.0   
11       2023-11-01           0.0        0.0        0.0        0.0   
12       2023-12-01           0.0        0.0        0.0        0.0   

            5      